# Full data

## Frequency based

In [2]:
%cd

/home/gyubin


/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/IPython/core/magics/osm.py:428: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import pandas as pd

df = pd.read_excel('./Knowledge_Editing_Dataset/data/one_hop/re_df_sbj_full_rel.xlsx')
#df = pd.read_excel("Knowledge_Editing_Dataset/df_one_hop_sbj_34.xlsx")
df.head()

,index,subject,subject_id,one_hop
0,0,Danielle Darrieux,Q234149,"Darrieux,Danielle Darrieux,Danielle Yvonne Mar..."
1,1,Edwin of Northumbria,Q348955,"male,Hatfield Chase,Cwenburh,Kingdom of Deira,..."
2,2,Toko Yasuda,Q7813654,"rock music,Blonde Redhead,plvsvltramusic,music..."
3,3,Autonomous University of Madrid,Q788091,"Spain,Coalition for Advancing Research Assessm..."
4,4,Lyon,Q6709677,"Lyon Township, Oakland County, Michigan,United..."


In [5]:
df.loc[7278, 'one_hop'].split(',')

['Tim Cook',
 'Timothy Donald Cook',
 'timcook',
 'Apple',
 'cookt',
 'English',
 'Tim',
 'Cook',
 'Donald',
 'Palo Alto',
 'Timothy',
 'Ice Bucket Challenge',
 'Lentapedia',
 'member of the Alabama Academy of Honor',
 'computer scientist',
 'businessperson',
 'Apple',
 'manager',
 'engineer',
 'Mobile',
 'human',
 'chief executive officer',
 'male',
 'presenter',
 'United States of America',
 'IBM',
 'computer industry',
 'Fuqua School of Business',
 'management',
 'Financial Times Person of the Year',
 'Duke University',
 'homosexuality',
 'Robertsdale High School',
 'Auburn University',
 'Rostrevor College']

In [6]:
from gensim.models import KeyedVectors

# 사전 학습된 Google News Word2Vec 모델을 다운로드
# 다운로드 링크: https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

model_path = 'Knowledge_Editing_Dataset/data/similarity/GoogleNews-vectors-negative300.bin.gz'
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [7]:
i = 7278
sub = df.loc[i,'subject']
#sub = "Messi"
full_hops = df.loc[i, 'one_hop'].split(',')
data = dict()
print(sub)


for word in full_hops:
    if sub in word2vec_model and word in word2vec_model:
        data[word] = word2vec_model.similarity(sub, word)


# 값을 기준으로 내림차순 정렬
sorted_data = dict(sorted(data.items(), key=lambda item: item[1], reverse=True))
#print(f'The similarity between "{sub}" and "{word}" is: {similarity}')
print(sorted_data)
for word in sorted_data:
    print(f'The similarity between "{sub}" and "{word}" is: {sorted_data[word]}')

Tim Cook
{}


In [9]:
import fasttext.util
import fasttext
import numpy as np

# 영어 모델 다운로드
fasttext.util.download_model('en', if_exists='ignore')  # 영어 모델 다운로드
ft = fasttext.load_model('cc.en.300.bin')  # 다운로드한 모델 로드

i = 7278
sub = df.loc[i, 'subject']
# sub = "Messi"
full_hops = df.loc[i, 'one_hop'].split(',')
data = dict()
print(sub)
print(full_hops)

# 두 벡터의 코사인 유사도를 계산하는 함수
def cosine_similarity(vec_a, vec_b):
    return np.dot(vec_a, vec_b) / (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))

for word in full_hops:
    # FastText 모델을 사용하여 단어 벡터를 가져옴
    sub_vector = ft.get_word_vector(sub)
    word_vector = ft.get_word_vector(word)
    
    # 두 단어 벡터 간의 유사도 계산
    if np.any(sub_vector) and np.any(word_vector):  # 벡터가 존재할 때만 계산
        similarity = cosine_similarity(sub_vector, word_vector)
        data[word] = similarity

# 값을 기준으로 내림차순 정렬
sorted_data = dict(sorted(data.items(), key=lambda item: item[1], reverse=True))
print(sorted_data)
for word in sorted_data:
    print(f'The similarity between "{sub}" and "{word}" is: {sorted_data[word]}')


Tim Cook
['Tim Cook', 'Timothy Donald Cook', 'timcook', 'Apple', 'cookt', 'English', 'Tim', 'Cook', 'Donald', 'Palo Alto', 'Timothy', 'Ice Bucket Challenge', 'Lentapedia', 'member of the Alabama Academy of Honor', 'computer scientist', 'businessperson', 'Apple', 'manager', 'engineer', 'Mobile', 'human', 'chief executive officer', 'male', 'presenter', 'United States of America', 'IBM', 'computer industry', 'Fuqua School of Business', 'management', 'Financial Times Person of the Year', 'Duke University', 'homosexuality', 'Robertsdale High School', 'Auburn University', 'Rostrevor College']
{'Tim Cook': 1.0000001, 'United States of America': 0.5730964, 'Timothy Donald Cook': 0.5358318, 'Financial Times Person of the Year': 0.49166813, 'Robertsdale High School': 0.47016907, 'chief executive officer': 0.45391336, 'Rostrevor College': 0.44805196, 'Duke University': 0.4102939, 'Auburn University': 0.39585412, 'Palo Alto': 0.36608255, 'member of the Alabama Academy of Honor': 0.34196532, 'Ice B

# Embedding based

In [10]:
from sentence_transformers import SentenceTransformer, util

# SBERT 모델 로드 (pre-trained 모델 사용)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 두 단어 정의
word1 = "apple"
word2 = "orange"

# 각 단어에 대한 임베딩 생성
embedding1 = model.encode(word1, convert_to_tensor=True)
embedding2 = model.encode(word2, convert_to_tensor=True)

# 코사인 유사도 계산
similarity = util.pytorch_cos_sim(embedding1, embedding2)

# 유사도 출력
print(f"'{word1}'와 '{word2}'의 유사도: {similarity.item()}")


'apple'와 'orange'의 유사도: 0.360235333442688


In [11]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

# SBERT 모델 로드 (pre-trained 모델 사용)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


def emb_2word(word1, word2):
    # 각 단어에 대한 임베딩 생성
    embedding1 = model.encode(word1, convert_to_tensor=True)
    embedding2 = model.encode(word2, convert_to_tensor=True)

    # 코사인 유사도 계산
    similarity = util.pytorch_cos_sim(embedding1, embedding2)
    return similarity.item()


i = 7278
sub = df.loc[i,'subject']
full_hops = df.loc[i, 'one_hop'].split(',')
full_hops = list(set(full_hops))
data = dict()
print(sub)
#print(hops)

for word in tqdm(full_hops):
    data[word] = emb_2word(sub, word)

# 값을 기준으로 내림차순 정렬
sorted_data = dict(sorted(data.items(), key=lambda item: item[1], reverse=True))
#print(f'The similarity between "{sub}" and "{word}" is: {similarity}')
print(sorted_data)
for word in sorted_data:
    print(f'The similarity between "{sub}" and "{word}" is: {sorted_data[word]}')


Tim Cook


100%|██████████| 34/34 [00:00<00:00, 44.18it/s]

{'Tim Cook': 1.0000001192092896, 'Cook': 0.8581556081771851, 'Timothy Donald Cook': 0.8197294473648071, 'cookt': 0.748644232749939, 'Tim': 0.508840799331665, 'manager': 0.4214176833629608, 'timcook': 0.3937060534954071, 'Timothy': 0.33772513270378113, 'presenter': 0.3251972198486328, 'businessperson': 0.3070620894432068, 'chief executive officer': 0.26448896527290344, 'engineer': 0.26199817657470703, 'male': 0.2525852620601654, 'computer scientist': 0.24279183149337769, 'management': 0.22923238575458527, 'Apple': 0.22832632064819336, 'English': 0.2205059975385666, 'Donald': 0.21591851115226746, 'Financial Times Person of the Year': 0.20169135928153992, 'Fuqua School of Business': 0.1741185486316681, 'human': 0.1693943589925766, 'computer industry': 0.13168072700500488, 'Palo Alto': 0.12846770882606506, 'Robertsdale High School': 0.0976412296295166, 'Lentapedia': 0.08979896456003189, 'member of the Alabama Academy of Honor': 0.07842639833688736, 'Rostrevor College': 0.04673917591571808,

In [12]:
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import torch
import torch.nn.functional as F

# BERT 모델과 토크나이저 로드
model_name = 'bert-base-uncased'  # 또는 다른 BERT 모델
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def emb_2word(word1, word2):
    # 각 단어에 대한 토큰화
    inputs1 = tokenizer(word1, return_tensors='pt', padding=True, truncation=True)
    inputs2 = tokenizer(word2, return_tensors='pt', padding=True, truncation=True)

    # 모델을 통해 임베딩 생성
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)

    # CLS 토큰의 임베딩 사용 (또는 평균 임베딩 사용 가능)
    embedding1 = outputs1.last_hidden_state[:, 0, :]
    embedding2 = outputs2.last_hidden_state[:, 0, :]

    # 코사인 유사도 계산
    similarity = F.cosine_similarity(embedding1, embedding2)
    return similarity.item()

i = 7278
sub = df.loc[i, 'subject']
full_hops = df.loc[i, 'one_hop'].split(',')
full_hops = list(set(full_hops))
data = dict()
print(sub)

for word in tqdm(full_hops):
    data[word] = emb_2word(sub, word)

# 값을 기준으로 내림차순 정렬
sorted_data = dict(sorted(data.items(), key=lambda item: item[1], reverse=True))

print(sorted_data)
for word in sorted_data:
    print(f'The similarity between "{sub}" and "{word}" is: {sorted_data[word]}')

/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Tim Cook


100%|██████████| 34/34 [00:01<00:00, 19.33it/s]

{'Tim Cook': 1.0, 'Timothy Donald Cook': 0.9850752949714661, 'Timothy': 0.9730494618415833, 'Tim': 0.9730461835861206, 'Cook': 0.9729781150817871, 'Donald': 0.9687259793281555, 'Apple': 0.9652021527290344, 'timcook': 0.9591405987739563, 'IBM': 0.9561740159988403, 'Lentapedia': 0.9495500922203064, 'Duke University': 0.9435211420059204, 'businessperson': 0.9332523941993713, 'cookt': 0.9331498146057129, 'presenter': 0.9315454363822937, 'Robertsdale High School': 0.9282682538032532, 'Mobile': 0.927878201007843, 'Auburn University': 0.9219095706939697, 'chief executive officer': 0.9154585599899292, 'Palo Alto': 0.9093027114868164, 'management': 0.8933460116386414, 'member of the Alabama Academy of Honor': 0.8887491822242737, 'human': 0.8844159245491028, 'homosexuality': 0.8837743401527405, 'English': 0.8828087449073792, 'male': 0.8799480199813843, 'United States of America': 0.8794611096382141, 'Ice Bucket Challenge': 0.8663344979286194, 'Financial Times Person of the Year': 0.8623477816581